# TOPSIS Method Implementation (Step-by-Step)

## Objective
To rank multiple alternatives based on multiple criteria using the
**TOPSIS (Technique for Order Preference by Similarity to Ideal Solution)** method.

The alternative closest to the ideal best and farthest from the ideal worst
is considered the best.


### Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np


### Load Input File (Handling File Not Found)

In [ ]:
try:
    df = pd.read_excel("data.xlsx") # make sure that file is in the same folder
except FileNotFoundError:
    print("Error: Input file not found.")
    raise
except Exception as e:
    print("Error while reading the input file:", e)
    raise


### Validate Minimum Column Requirement

In [ ]:
if df.shape[1] < 3:
    raise ValueError("Input file must contain at least three columns.")


### Separate Identifier and Criteria Columns

In [ ]:
names = df.iloc[:, 0]
data = df.iloc[:, 1:]

# TOPSIS works only on numeric criteria

### Validate Numeric Criteria

In [ ]:
try:
    data = data.astype(float)
except ValueError:
    raise ValueError("From 2nd to last columns must contain numeric values only.")


### Define Weights and Impacts

In [ ]:
weights = np.array([1, 1, 1, 1, 1])
impacts = ['+', '+', '-', '+', '+']

#Weights → importance of criteria
# Impacts:
#  + → higher is better
#  - → lower is better

### Validate Weights & Impacts

In [ ]:
if len(weights) != data.shape[1]:
    raise ValueError("Number of weights must match number of criteria.")

if len(impacts) != data.shape[1]:
    raise ValueError("Number of impacts must match number of criteria.")

for i in impacts:
    if i not in ['+', '-']:
        raise ValueError("Impacts must be either '+' or '-'.")


### Normalize the Decision Matrix

In [ ]:
norm_data = data / np.sqrt((data ** 2).sum())


### Apply Weights

In [ ]:
weighted_data = norm_data * weights


### Determine Ideal Best & Ideal Worst

In [ ]:
ideal_best = []
ideal_worst = []

for i in range(len(impacts)):
    if impacts[i] == '+':
        ideal_best.append(weighted_data.iloc[:, i].max())
        ideal_worst.append(weighted_data.iloc[:, i].min())
    else:
        ideal_best.append(weighted_data.iloc[:, i].min())
        ideal_worst.append(weighted_data.iloc[:, i].max())

ideal_best = np.array(ideal_best)
ideal_worst = np.array(ideal_worst)


### Calculate Distance from Ideal Solutions

In [ ]:
dist_best = np.sqrt(((weighted_data - ideal_best) ** 2).sum(axis=1))
dist_worst = np.sqrt(((weighted_data - ideal_worst) ** 2).sum(axis=1))


### Calculate TOPSIS Score

In [ ]:
topsis_score = dist_worst / (dist_best + dist_worst)


### Rank the Alternatives

In [ ]:
df['Topsis Score'] = topsis_score
df['Rank'] = df['Topsis Score'].rank(ascending=False).astype(int)

df


,Fund Name,P1,P2,P3,P4,P5,Topsis Score,Rank
0,M1,0.84,0.71,6.7,42.1,12.59,0.382109,6
1,M2,0.91,0.83,7.0,31.7,10.11,0.366492,7
2,M3,0.79,0.62,4.8,46.7,13.23,0.496361,4
3,M4,0.78,0.61,6.4,42.4,12.55,0.324792,8
4,M5,0.94,0.88,3.6,62.2,16.91,0.972128,1
5,M6,0.88,0.77,6.5,51.5,14.91,0.547048,3
6,M7,0.66,0.44,5.3,48.9,13.83,0.395015,5
7,M8,0.93,0.86,3.4,37.0,10.55,0.560092,2


### Save Result File

In [ ]:
df.to_csv("topsis_result.csv", index=False)
